<a href="https://colab.research.google.com/github/totiela/vk-ml-time-series-test-task/blob/main/notebooks/validation_and_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Импорт библиотек и загрузка данных

In [1]:
# загрузка необходимых библиотек
!pip install -q catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.8 MB/s eta 0:00:00


In [2]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
import joblib
import warnings
from sklearn.model_selection import StratifiedKFold
from catboost import Pool, CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score

# Отключение всех предупреждений
warnings.filterwarnings('ignore')

In [3]:
# Клонируем репозитоиий
!git clone https://github.com/totiela/vk-ml-time-series-test-task.git
%cd vk-ml-time-series-test-task

Cloning into 'vk-ml-time-series-test-task'...
remote: Enumerating objects: 121, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 121 (delta 18), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (121/121), 47.80 MiB | 9.22 MiB/s, done.
Resolving deltas: 100% (18/18), done.
/content/vk-ml-time-series-test-task


In [4]:
# Загрузка тренировочных данных с Google Диска (из-за большого объема файлов)

# Загрузка первого файла
!gdown --id 1L0ACJIPLvyMh-KN6NXuBF6FDMQu5fXVz -O train_141_features.parquet

# Загрузка второго файла
!gdown --id 1Q-5MCMIvdG4N9yij1pJLgXcEMwALEt1Y -O train_116_features.parquet

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1L0ACJIPLvyMh-KN6NXuBF6FDMQu5fXVz
To: /content/vk-ml-time-series-test-task/train_141_features.parquet
100% 73.3M/73.3M [00:00<00:00, 177MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1Q-5MCMIvdG4N9yij1pJLgXcEMwALEt1Y
To: /content/vk-ml-time-series-test-task/train_116_features.parquet
100% 59.0M/59.0M [00:00<00:00, 62.5MB/s]


In [5]:
# Инициализация исходного датасета
train_df = pd.read_parquet('data/raw/train.parquet')
y = train_df['label']

In [6]:
# Инициализация извлеченных фичей
train_116_features = pd.read_parquet('train_116_features.parquet')
train_141_features = pd.read_parquet('train_141_features.parquet')

In [7]:
# Проверка данных
train_116_features.head()

,dates_count,close_to_zero_diff,quarterly_mean,winter_percent,summer_percent,zero_crossings,rolling_mean_3,rolling_mean_6,rolling_mean_12,rolling_std_12,...,value__number_crossing_m__m_0,value__energy_ratio_by_chunks__num_segments_10__segment_focus_8,value__energy_ratio_by_chunks__num_segments_10__segment_focus_9,value__count_below__t_0,value__lempel_ziv_complexity__bins_2,value__lempel_ziv_complexity__bins_5,value__lempel_ziv_complexity__bins_10,value__lempel_ziv_complexity__bins_100,value__permutation_entropy__dimension_7__tau_1,value__mean_n_absolute_max__number_of_maxima_7
0,62,2.0,0.1200,27.419355,24.193548,31,-0.243333,0.180000,-0.225833,0.874606,...,28.0,0.054173,0.027211,0.548387,0.290323,0.387097,0.500000,0.774194,4.025352,2.905714
1,45,1.0,-1.1500,24.444444,26.666667,16,-0.151076,-0.401076,-0.754409,1.367154,...,16.0,0.040909,0.072168,0.622222,0.355556,0.488889,0.533333,0.866667,3.663562,3.067604
2,48,3.0,0.4825,25.000000,25.000000,9,-0.030000,0.045000,-0.499167,0.824692,...,9.0,0.146157,0.008721,0.562500,0.333333,0.416667,0.562500,0.833333,3.737670,1.988571
3,48,5.0,0.1975,25.000000,25.000000,11,1.166667,1.018333,1.178333,0.386166,...,11.0,0.051597,0.083085,0.541667,0.312500,0.458333,0.583333,0.854167,3.704663,2.064286
4,56,0.0,0.5550,25.000000,26.785714,25,-1.646667,-0.438333,-0.367500,1.306085,...,25.0,0.053602,0.134183,0.642857,0.303571,0.464286,0.553571,0.803571,3.912023,2.347143


In [8]:
# Проверка данных
train_141_features.head()

,dates_count,close_to_zero_diff,quarterly_mean,winter_percent,summer_percent,zero_crossings,rolling_mean_3,rolling_mean_6,rolling_mean_12,rolling_std_12,...,value__energy_ratio_by_chunks__num_segments_10__segment_focus_9,value__count_above__t_0,value__lempel_ziv_complexity__bins_2,value__lempel_ziv_complexity__bins_3,value__lempel_ziv_complexity__bins_5,value__lempel_ziv_complexity__bins_10,value__lempel_ziv_complexity__bins_100,value__permutation_entropy__dimension_6__tau_1,value__permutation_entropy__dimension_7__tau_1,value__mean_n_absolute_max__number_of_maxima_7
0,62,2.0,0.1200,27.419355,24.193548,31,-0.243333,0.180000,-0.225833,0.874606,...,0.027211,0.483871,0.290323,0.322581,0.387097,0.500000,0.774194,4.018730,4.025352,2.905714
1,45,1.0,-1.1500,24.444444,26.666667,16,-0.151076,-0.401076,-0.754409,1.367154,...,0.072168,0.377778,0.355556,0.400000,0.488889,0.533333,0.866667,3.654222,3.663562,3.067604
2,48,3.0,0.4825,25.000000,25.000000,9,-0.030000,0.045000,-0.499167,0.824692,...,0.008721,0.437500,0.333333,0.395833,0.416667,0.562500,0.833333,3.761200,3.737670,1.988571
3,48,5.0,0.1975,25.000000,25.000000,11,1.166667,1.018333,1.178333,0.386166,...,0.083085,0.458333,0.312500,0.375000,0.458333,0.583333,0.854167,3.684553,3.704663,2.064286
4,56,0.0,0.5550,25.000000,26.785714,25,-1.646667,-0.438333,-0.367500,1.306085,...,0.134183,0.357143,0.303571,0.375000,0.464286,0.553571,0.803571,3.904643,3.912023,2.347143


# Валидация

In [50]:
# Функция для стратифицированной валидации модели с блендингом 4-х моделей

def stratified_kfold_blending(X_116, X_141, y, n_splits=5, random_seed=42):
    """
    Проводит стратифицированную кросс-валидацию для ансамбля моделей и вычисляет итоговый скор с использованием блендинга.

    Параметры:
    - X_116 (pd.DataFrame): Признаки для моделей с гиперпараметрами под 116 признаков.
    - X_141 (pd.DataFrame): Признаки для моделей с гиперпараметрами под 141 признак.
    - y (pd.Series): Целевая переменная.
    - n_splits (int): Количество фолдов для кросс-валидации. По умолчанию 5.
    - random_seed (int): Случайное зерно для повторяемости. По умолчанию 42.

    Используемые модели:
    - cb_tuned_116: CatBoost с тюнингом для 116 признаков.
    - xgb_tuned_116: XGBoost с тюнингом для 116 признаков.
    - cb_no_tuned_141: CatBoost с фиксированными гиперпараметрами для 141 признака.
    - xgb_no_tuned_141: XGBoost с фиксированными гиперпараметрами для 141 признака.

    Стратегия блендинга:
    Веса моделей заданы для расчета итогового прогноза:
    - w_cb_tuned = 0.3
    - w_xgb_tuned = 0.3
    - w_cb_no_tuned = 0.3
    - w_xgb_no_tuned = 0.1

    Логи:
    - AUC каждой модели по фолдам и их среднее значение.
    - Итоговый скор бленда по каждому фолду и его среднее значение.

    Возвращает:
    - tuple: Среднее значение AUC и стандартное отклонение бленда по всем фолдам.
    """

    # Заданные веса для моделей в блендинге
    weights = {
        'w_cb_tuned': 0.3,
        'w_xgb_tuned': 0.3,
        'w_cb_no_tuned': 0.3,
        'w_xgb_no_tuned': 0.1
    }

    # Инициализация для хранения результатов
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_seed)
    scores = {
        'cb_tuned_116': [],
        'xgb_tuned_116': [],
        'cb_no_tuned_141': [],
        'xgb_no_tuned_141': [],
        'blended': []
    }

    # Настроенные параметры для каждой модели
    cb_tuned_116_params = {
        'learning_rate': 0.04035489782502211,
        'l2_leaf_reg': 48.97692771208465,
        'depth': 9,
        'auto_class_weights': 'SqrtBalanced',
        'bootstrap_type': 'Bernoulli',
        'boosting_type': 'Ordered',
        'subsample': 0.8792927677279381,
        'thread_count': -1,
        'random_seed': 42,
        'eval_metric': 'AUC',
        'task_type': 'GPU',
        'iterations': 1410
    }
    xgb_tuned_116_params = {
        'learning_rate': 0.021730647774665324,
        'max_depth': 6,
        'min_child_weight': 0.014400926926567095,
        'gamma': 0.013054710551594132,
        'subsample': 0.8309621984412547,
        'colsample_bytree': 0.6342723560146742,
        'reg_alpha': 3.1087857145941866,
        'reg_lambda': 0.007273323924804515,
        'tree_method': 'hist',
        'device': 'cuda',
        'eval_metric': 'auc',
        'n_estimators': 1574
    }

    # Модели с фиксированными гиперпараметрами
    cb_no_tuned_141 = CatBoostClassifier(
        learning_rate=0.025682, thread_count=-1, random_seed=42,
        eval_metric='AUC', task_type='GPU', iterations=4149
    )
    xgb_no_tuned_141 = XGBClassifier(
        learning_rate=0.035, tree_method='hist', device='cuda',
        eval_metric='auc', n_estimators=1204
    )

    # Проход по фолдам
    for fold, (train_idx, val_idx) in enumerate(skf.split(X_116, y)):
        print(f"\n---------- Fold {fold + 1}/{n_splits} ----------")

        # Разделение данных
        X_train_116, X_val_116 = X_116.iloc[train_idx], X_116.iloc[val_idx]
        X_train_141, X_val_141 = X_141.iloc[train_idx], X_141.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        # Пулы для CatBoost
        train_pool_116 = Pool(data=X_train_116, label=y_train)
        val_pool_116 = Pool(data=X_val_116, label=y_val)
        train_pool_141 = Pool(data=X_train_141, label=y_train)
        val_pool_141 = Pool(data=X_val_141, label=y_val)

        # CatBoost с тюнингом
        cb_tuned_116 = CatBoostClassifier(**cb_tuned_116_params)
        cb_tuned_116.fit(train_pool_116, eval_set=val_pool_116, verbose=100)
        cb_tuned_116_pred = cb_tuned_116.predict_proba(X_val_116)[:, 1]
        cb_tuned_116_score = roc_auc_score(y_val, cb_tuned_116_pred)
        scores['cb_tuned_116'].append(cb_tuned_116_score)
        print(f"cb_tuned_116 AUC: {cb_tuned_116_score:.4f}")

        # XGBoost с тюнингом
        xgb_tuned_116 = XGBClassifier(**xgb_tuned_116_params)
        xgb_tuned_116.fit(X_train_116, y_train, eval_set=[(X_val_116, y_val)], verbose=100)
        xgb_tuned_116_pred = xgb_tuned_116.predict_proba(X_val_116)[:, 1]
        xgb_tuned_116_score = roc_auc_score(y_val, xgb_tuned_116_pred)
        scores['xgb_tuned_116'].append(xgb_tuned_116_score)
        print(f"xgb_tuned_116 AUC: {xgb_tuned_116_score:.4f}")

        # CatBoost без тюнинга
        cb_no_tuned_141.fit(train_pool_141, eval_set=val_pool_141, verbose=100)
        cb_no_tuned_141_pred = cb_no_tuned_141.predict_proba(X_val_141)[:, 1]
        cb_no_tuned_141_score = roc_auc_score(y_val, cb_no_tuned_141_pred)
        scores['cb_no_tuned_141'].append(cb_no_tuned_141_score)
        print(f"cb_no_tuned_141 AUC: {cb_no_tuned_141_score:.4f}")

        # XGBoost без тюнинга
        xgb_no_tuned_141.fit(X_train_141, y_train, eval_set=[(X_val_141, y_val)], verbose=100)
        xgb_no_tuned_141_pred = xgb_no_tuned_141.predict_proba(X_val_141)[:, 1]
        xgb_no_tuned_141_score = roc_auc_score(y_val, xgb_no_tuned_141_pred)
        scores['xgb_no_tuned_141'].append(xgb_no_tuned_141_score)
        print(f"xgb_no_tuned_141 AUC: {xgb_no_tuned_141_score:.4f}")

        # Блендинг прогнозов
        blended_pred = (
            weights['w_cb_tuned'] * cb_tuned_116_pred +
            weights['w_xgb_tuned'] * xgb_tuned_116_pred +
            weights['w_cb_no_tuned'] * cb_no_tuned_141_pred +
            weights['w_xgb_no_tuned'] * xgb_no_tuned_141_pred
        )
        blended_score = roc_auc_score(y_val, blended_pred)
        scores['blended'].append(blended_score)
        print(f"Blended AUC: {blended_score:.4f}")

    # Итоговый отчет
    print("\n---------- Итоговые результаты ----------")
    for model_name, model_scores in scores.items():
        mean_score = np.mean(model_scores)
        std_score = np.std(model_scores)
        print(f"{model_name} Средний AUC: {mean_score:.4f} ± {std_score:.4f}")

    # Итоговый возврат среднего скора бленда
    mean_blended_score = np.mean(scores['blended'])
    std_blended_score = np.std(scores['blended'])
    return mean_blended_score, std_blended_score

In [51]:
# Вызов функции для валидации
mean_blended_score, std_blended_score = stratified_kfold_blending(train_116_features, train_141_features, y)


---------- Fold 1/5 ----------


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.8951868	best: 0.8951868 (0)	total: 413ms	remaining: 9m 41s
100:	test: 0.9341703	best: 0.9341703 (100)	total: 26.4s	remaining: 5m 41s
200:	test: 0.9393543	best: 0.9393543 (200)	total: 49.4s	remaining: 4m 57s
300:	test: 0.9415950	best: 0.9415950 (300)	total: 1m 13s	remaining: 4m 29s
400:	test: 0.9429470	best: 0.9429470 (400)	total: 1m 38s	remaining: 4m 8s
500:	test: 0.9435159	best: 0.9435159 (500)	total: 2m 3s	remaining: 3m 44s
600:	test: 0.9439718	best: 0.9439718 (600)	total: 2m 29s	remaining: 3m 20s
700:	test: 0.9442640	best: 0.9442735 (695)	total: 2m 54s	remaining: 2m 56s
800:	test: 0.9445614	best: 0.9445614 (800)	total: 3m 19s	remaining: 2m 31s
900:	test: 0.9447801	best: 0.9447801 (900)	total: 3m 44s	remaining: 2m 7s
1000:	test: 0.9448231	best: 0.9448397 (960)	total: 4m 10s	remaining: 1m 42s
1100:	test: 0.9448434	best: 0.9448944 (1075)	total: 4m 35s	remaining: 1m 17s
1200:	test: 0.9447340	best: 0.9448944 (1075)	total: 5m	remaining: 52.3s
1300:	test: 0.9447623	best: 0.94489

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.8835203	best: 0.8835203 (0)	total: 14ms	remaining: 58.2s
100:	test: 0.9245166	best: 0.9245166 (100)	total: 701ms	remaining: 28.1s
200:	test: 0.9320006	best: 0.9320006 (200)	total: 1.39s	remaining: 27.4s
300:	test: 0.9356626	best: 0.9356626 (300)	total: 2.07s	remaining: 26.5s
400:	test: 0.9378734	best: 0.9378734 (400)	total: 3.33s	remaining: 31.2s
500:	test: 0.9392576	best: 0.9392576 (500)	total: 5.68s	remaining: 41.3s
600:	test: 0.9402514	best: 0.9402514 (600)	total: 6.38s	remaining: 37.6s
700:	test: 0.9409776	best: 0.9409776 (700)	total: 7.03s	remaining: 34.6s
800:	test: 0.9415651	best: 0.9415651 (800)	total: 7.72s	remaining: 32.3s
900:	test: 0.9420352	best: 0.9420352 (900)	total: 8.39s	remaining: 30.3s
1000:	test: 0.9423858	best: 0.9423858 (1000)	total: 9.05s	remaining: 28.5s
1100:	test: 0.9426274	best: 0.9426274 (1100)	total: 9.74s	remaining: 27s
1200:	test: 0.9429405	best: 0.9429405 (1200)	total: 10.4s	remaining: 25.6s
1300:	test: 0.9432203	best: 0.9432203 (1300)	total: 

...

0:	test: 0.8789007	best: 0.8789007 (0)	total: 9.08ms	remaining: 37.7s
100:	test: 0.9214078	best: 0.9214078 (100)	total: 750ms	remaining: 30.1s
200:	test: 0.9289654	best: 0.9289654 (200)	total: 1.44s	remaining: 28.3s
300:	test: 0.9323996	best: 0.9323996 (300)	total: 2.13s	remaining: 27.3s
400:	test: 0.9345086	best: 0.9345086 (400)	total: 2.8s	remaining: 26.2s
500:	test: 0.9358963	best: 0.9358963 (500)	total: 3.48s	remaining: 25.4s
600:	test: 0.9368799	best: 0.9368799 (600)	total: 4.17s	remaining: 24.6s
700:	test: 0.9375079	best: 0.9375079 (700)	total: 4.83s	remaining: 23.8s
800:	test: 0.9380746	best: 0.9380746 (800)	total: 5.52s	remaining: 23.1s
900:	test: 0.9384876	best: 0.9384876 (900)	total: 6.2s	remaining: 22.4s
1000:	test: 0.9388222	best: 0.9388222 (1000)	total: 6.87s	remaining: 21.6s
1100:	test: 0.9391366	best: 0.9391366 (1100)	total: 9.42s	remaining: 26.1s
1200:	test: 0.9394372	best: 0.9394372 (1200)	total: 10.3s	remaining: 25.4s
1300:	test: 0.9397117	best: 0.9397117 (1300)	total

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.8937630	best: 0.8937630 (0)	total: 275ms	remaining: 6m 28s
100:	test: 0.9314843	best: 0.9314843 (100)	total: 25.1s	remaining: 5m 24s
200:	test: 0.9370547	best: 0.9370547 (200)	total: 49s	remaining: 4m 55s
300:	test: 0.9392788	best: 0.9392788 (300)	total: 1m 12s	remaining: 4m 26s
400:	test: 0.9407064	best: 0.9407064 (400)	total: 1m 37s	remaining: 4m 5s
500:	test: 0.9415414	best: 0.9415414 (500)	total: 2m 2s	remaining: 3m 42s
600:	test: 0.9419952	best: 0.9419952 (600)	total: 2m 28s	remaining: 3m 20s
700:	test: 0.9423192	best: 0.9423192 (700)	total: 2m 53s	remaining: 2m 55s
800:	test: 0.9426380	best: 0.9426380 (800)	total: 3m 19s	remaining: 2m 31s
900:	test: 0.9426718	best: 0.9426718 (900)	total: 3m 45s	remaining: 2m 7s
1000:	test: 0.9429059	best: 0.9429059 (1000)	total: 4m 11s	remaining: 1m 42s
1100:	test: 0.9430314	best: 0.9430408 (1095)	total: 4m 36s	remaining: 1m 17s
1200:	test: 0.9430754	best: 0.9430801 (1195)	total: 5m 1s	remaining: 52.5s
1300:	test: 0.9431655	best: 0.943

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.8871548	best: 0.8871548 (0)	total: 8.83ms	remaining: 36.6s
100:	test: 0.9211902	best: 0.9211902 (100)	total: 714ms	remaining: 28.6s
200:	test: 0.9286316	best: 0.9286316 (200)	total: 1.41s	remaining: 27.7s
300:	test: 0.9327928	best: 0.9327928 (300)	total: 2.13s	remaining: 27.3s
400:	test: 0.9351690	best: 0.9351690 (400)	total: 2.8s	remaining: 26.2s
500:	test: 0.9367884	best: 0.9367884 (500)	total: 3.83s	remaining: 27.9s
600:	test: 0.9377913	best: 0.9377913 (600)	total: 6.44s	remaining: 38s
700:	test: 0.9385667	best: 0.9385667 (700)	total: 7.14s	remaining: 35.1s
800:	test: 0.9392816	best: 0.9392816 (800)	total: 7.82s	remaining: 32.7s
900:	test: 0.9398355	best: 0.9398355 (900)	total: 8.5s	remaining: 30.7s
1000:	test: 0.9402660	best: 0.9402660 (1000)	total: 9.16s	remaining: 28.8s
1100:	test: 0.9406601	best: 0.9406601 (1100)	total: 9.86s	remaining: 27.3s
1200:	test: 0.9410254	best: 0.9410254 (1200)	total: 10.5s	remaining: 25.9s
1300:	test: 0.9413489	best: 0.9413489 (1300)	total: 

# Обучение модели

In [36]:

def train_and_return_models(X_116, X_141, y):
    """
    Обучает четыре модели и возвращает их вместе с функцией для блендирования предсказаний.

    Параметры:
    - X_116 (pd.DataFrame): Признаки для моделей с гиперпараметрами под 116 признаков.
    - X_141 (pd.DataFrame): Признаки для моделей с гиперпараметрами под 141 признак.
    - y (pd.Series): Целевая переменная.

    Возвращает:
    - cb_tuned_116: Обученная модель CatBoost с гиперпараметрами для 116 признаков.
    - xgb_tuned_116: Обученная модель XGBoost с гиперпараметрами для 116 признаков.
    - cb_no_tuned_141: Обученная модель CatBoost без гиперпараметров для 141 признака.
    - xgb_no_tuned_141: Обученная модель XGBoost без гиперпараметров для 141 признака.
    - blended_predict: Функция для блендирования предсказаний на основе четырех моделей.
    """

    # Настроенные параметры для моделей
    cb_tuned_116_params = {
        'learning_rate': 0.04035489782502211,
        'l2_leaf_reg': 48.97692771208465,
        'depth': 9,
        'auto_class_weights': 'SqrtBalanced',
        'bootstrap_type': 'Bernoulli',
        'boosting_type': 'Ordered',
        'subsample': 0.8792927677279381,
        'thread_count': -1,
        'random_seed': 42,
        'eval_metric': 'AUC',
        'task_type': 'GPU',
        'iterations': 1410
    }
    xgb_tuned_116_params = {
        'learning_rate': 0.021730647774665324,
        'max_depth': 6,
        'min_child_weight': 0.014400926926567095,
        'gamma': 0.013054710551594132,
        'subsample': 0.8309621984412547,
        'colsample_bytree': 0.6342723560146742,
        'reg_alpha': 3.1087857145941866,
        'reg_lambda': 0.007273323924804515,
        'tree_method': 'hist',
        'device': 'cuda',
        'eval_metric': 'auc',
        'n_estimators': 1574
    }

    # Обучаем модели
    cb_tuned_116 = CatBoostClassifier(**cb_tuned_116_params)
    cb_tuned_116.fit(Pool(data=X_116, label=y), verbose=100)

    xgb_tuned_116 = XGBClassifier(**xgb_tuned_116_params)
    xgb_tuned_116.fit(X_116, y)

    cb_no_tuned_141 = CatBoostClassifier(
        learning_rate=0.025682, thread_count=-1, random_seed=42,
        eval_metric='AUC', task_type='GPU', iterations=4149
    )
    cb_no_tuned_141.fit(Pool(data=X_141, label=y), verbose=100)

    xgb_no_tuned_141 = XGBClassifier(
        learning_rate=0.035, tree_method='hist', device='cuda',
        eval_metric='auc', n_estimators=1204
    )
    xgb_no_tuned_141.fit(X_141, y)

    # Функция для блендирования предсказаний
    def blended_predict(X_116, X_141):
        cb_tuned_116_pred = cb_tuned_116.predict_proba(X_116)[:, 1]
        xgb_tuned_116_pred = xgb_tuned_116.predict_proba(X_116)[:, 1]
        cb_no_tuned_141_pred = cb_no_tuned_141.predict_proba(X_141)[:, 1]
        xgb_no_tuned_141_pred = xgb_no_tuned_141.predict_proba(X_141)[:, 1]

        # Блендирование предсказаний с весами
        blended_pred = (
            0.3 * cb_tuned_116_pred +
            0.3 * xgb_tuned_116_pred +
            0.3 * cb_no_tuned_141_pred +
            0.1 * xgb_no_tuned_141_pred
        )
        return blended_pred

    print("Модели обучены и готовы к использованию.")
    return cb_tuned_116, xgb_tuned_116, cb_no_tuned_141, xgb_no_tuned_141, blended_predict


In [37]:
# Использование функции для обучения и получения обученных моделей
cb_tuned_116, xgb_tuned_116, cb_no_tuned_141, xgb_no_tuned_141, blended_predict = train_and_return_models(train_116_features, train_141_features, y)

Default metric period is 5 because AUC is/are not implemented for GPU


0:	total: 356ms	remaining: 8m 22s
100:	total: 26.1s	remaining: 5m 38s
200:	total: 50s	remaining: 5m
300:	total: 1m 13s	remaining: 4m 30s
400:	total: 1m 38s	remaining: 4m 7s
500:	total: 2m 3s	remaining: 3m 43s
600:	total: 2m 29s	remaining: 3m 20s
700:	total: 2m 53s	remaining: 2m 55s
800:	total: 3m 19s	remaining: 2m 31s
900:	total: 3m 44s	remaining: 2m 6s
1000:	total: 4m 9s	remaining: 1m 41s
1100:	total: 4m 35s	remaining: 1m 17s
1200:	total: 5m	remaining: 52.3s
1300:	total: 5m 26s	remaining: 27.3s
1400:	total: 5m 51s	remaining: 2.26s
1409:	total: 5m 54s	remaining: 0us


Default metric period is 5 because AUC is/are not implemented for GPU


0:	total: 7.81ms	remaining: 32.4s
100:	total: 1.4s	remaining: 56.2s
200:	total: 3.86s	remaining: 1m 15s
300:	total: 4.56s	remaining: 58.3s
400:	total: 5.22s	remaining: 48.8s
500:	total: 5.9s	remaining: 42.9s
600:	total: 6.62s	remaining: 39.1s
700:	total: 7.27s	remaining: 35.7s
800:	total: 7.95s	remaining: 33.2s
900:	total: 8.73s	remaining: 31.5s
1000:	total: 11.3s	remaining: 35.5s
1100:	total: 12.3s	remaining: 34s
1200:	total: 13s	remaining: 31.9s
1300:	total: 13.7s	remaining: 30s
1400:	total: 14.4s	remaining: 28.1s
1500:	total: 15s	remaining: 26.5s
1600:	total: 15.7s	remaining: 24.9s
1700:	total: 16.3s	remaining: 23.5s
1800:	total: 17s	remaining: 22.2s
1900:	total: 17.7s	remaining: 20.9s
2000:	total: 18.3s	remaining: 19.7s
2100:	total: 19s	remaining: 18.5s
2200:	total: 19.6s	remaining: 17.4s
2300:	total: 20.3s	remaining: 16.3s
2400:	total: 21s	remaining: 15.3s
2500:	total: 21.7s	remaining: 14.3s
2600:	total: 24.7s	remaining: 14.7s
2700:	total: 25.5s	remaining: 13.7s
2800:	total: 26.1s

In [46]:
# Сохранение моделей
joblib.dump(cb_tuned_116, 'cb_tuned_116.pkl')
joblib.dump(xgb_tuned_116, 'xgb_tuned_116.pkl')
joblib.dump(cb_no_tuned_141, 'cb_no_tuned_141.pkl')
joblib.dump(xgb_no_tuned_141, 'xgb_no_tuned_141.pkl')

print("Модели успешно сохранены.")

Модели успешно сохранены.
